# Finding Polarity 

In [4]:
import nltk
import csv
import numpy as np
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [5]:
def read_csv_column(file_path, row_start, row_end):
    data = []
    rowcount = 0
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if rowcount >= row_start and rowcount < row_end:
                if len(row) > 7:
                    data.append(row[7]) # column 7 is the column for utterances
                    rowcount += 1
            elif rowcount >= row_end:
                break
            else: 
                rowcount+= 1
    return data

In [6]:
data = read_csv_column("/Users/tvidyala/Desktop/CHILDES/eng-NA-all-utterances.csv", 1, 5)

In [3]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
data = ["I feel good"]

for item in data:
    print(f"Text: {item}")
    result = sentiment_pipeline(item)
    print(f"Sentiment: {result[0]['label']}, Confidence: {result[0]['score']:.4f}\n")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


NameError: name 'tokenizer' is not defined

In [22]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
data = data 
for item in data:
    print(item)
    print(f"{analyzer.polarity_scores(item)}\n")

wanna stack them for me
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

that baby
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

oh
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

did you say baby
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}



In [1]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
data = ["i feel happy"] 
for item in data:
    print(item)
    print(f"{sentiment_pipeline(item)}\n")
    print(f"{analyzer.polarity_scores(item)}\n")

i feel happy


NameError: name 'sentiment_pipeline' is not defined

# Finding Valence, Arousal, and Dominance

In [ ]:
def VAD(text, vad_scores):
    i, j = 0, 0
    text_vad = np.zeros([3,])
    for word in text.split(' '):
        neg = 1  # reverse polarity for this word
        if word in vad_scores.index:
            if 'no' in text.split(' ')[max(0, j-6):j] or 'not' in text.split(' ')[max(0, j-6):j] or 'n\'t' in str(text.split(' ')[max(0, j-3):j]):
                neg = -1
            text_vad = vad_scores.loc[word] * neg + text_vad
            i += 1
        j += 1
    return text_vad / i

def read_csv_column(file_path, row_start, row_end, column_index):
    data = []
    rowcount = 0
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if rowcount >= row_start and rowcount < row_end:
                if len(row) > column_index:
                    data.append(row[column_index])
                rowcount += 1
            elif rowcount >= row_end:
                break
            else: 
                rowcount += 1
    return data

# Read child's data from CSV file
data = read_csv_column("/Users/tvidyala/Desktop/CHILDES/Laura.csv", 1, 10, 8) 
speaker_data = read_csv_column("/Users/tvidyala/Desktop/CHILDES/Laura.csv", 1, 10, 4) 
filename_data = read_csv_column("/Users/tvidyala/Desktop/CHILDES/Laura.csv", 1, 10, 0)

# Load VAD scores
vad_scores = pd.read_csv("vad-nrc.csv", index_col='Word')

# Calculate VAD scores for each CHILD utterance
for text in data:
    vad = VAD(text, vad_scores)
    print(data.index(text)+1)
    print(f'{speaker_data[data.index(text)]}: "{text}"')
    if np.isnan(vad).any():
        print("N/A \n")
    else:
        print(vad)
        print("")
        
arousal_scores = []
valence_scores = []
dominance_scores = []

for text in data:
    vad = VAD(text, vad_scores)
    arousal_scores.append(vad[0])
    valence_scores.append(vad[1])
    dominance_scores.append(vad[2])

df = pd.DataFrame({
    'File Name': filename_data,
    'Utterance': data,
    'Speaker': speaker_data,
    'Arousal': arousal_scores,
    'Valence': valence_scores,
    'Dominance': dominance_scores
})

df.to_csv("/Users/tvidyala/Desktop/CHILDES/LauraVADScores.csv", index=False) # updates LauraVADScores with VAD Scores


# Finding Average Valence, Arousal, and Dominance for Child by Age

In [2]:
def VAD(text, vad_scores):
    i, j = 0, 0
    text_vad = np.zeros([3,])
    for word in text.split(' '):
        neg = 1  # reverse polarity for this word
        if word in vad_scores.index:
            if 'no' in text.split(' ')[max(0, j-6):j] or 'not' in text.split(' ')[max(0, j-6):j] or 'n\'t' in str(text.split(' ')[max(0, j-3):j]):
                neg = -1
            text_vad = vad_scores.loc[word] * neg + text_vad
            i += 1
        j += 1
    return text_vad / i

def read_csv_column(file_path, row_start, row_end, column_index):
    data = []
    rowcount = 0
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if rowcount >= row_start and rowcount < row_end:
                if len(row) > column_index:
                    data.append(row[column_index])
                rowcount += 1
            elif rowcount >= row_end:
                break
            else: 
                rowcount += 1
    return data

# Read child's data from CSV file
data = read_csv_column("/Users/tvidyala/Desktop/CHILDES/Laura.csv", 1, 100, 8)  # Assuming column 7 contains utterances
speaker_data = read_csv_column("/Users/tvidyala/Desktop/CHILDES/Laura.csv", 1, 100, 4)  # Assuming column 3 contains speaker data
age_data = read_csv_column("/Users/tvidyala/Desktop/CHILDES/Laura.csv", 1, 100, 1)  # Assuming column 11 contains age data

# Load VAD scores
vad_scores = pd.read_csv("vad-nrc.csv", index_col='Word')

# Initialize variables to accumulate VAD scores
total_vad = np.zeros([3,])
valid_utterances = 0

# Calculate VAD scores for each CHILD utterance and accumulate

age = "105" # first number is age in years, second two numbers are number of months (01, 02, 03... 10, 11)
for text in data:
    if int(age) == int(age_data[data.index(text)][:3]): # fix this  
        vad = VAD(text, vad_scores)
        if speaker_data[data.index(text)] == "CHI":
            if not np.isnan(vad).any():
                total_vad += vad
                valid_utterances += 1

# Calculate the average VAD scores
average_vad = total_vad / valid_utterances

if age[1] == "0":
    print(f"Average VAD scores for Child's Utterances at {age[0]} year(s) and {age[2]} month(s)")
    print("Valence:", average_vad[0])
    print("Arousal:", average_vad[1])
    print("Dominance:", average_vad[2])
else: 
    print(f"Average VAD scores for Child's Utterances at {age[0]} year(s) and {age[1]+age[2]} month(s)")
    print("Valence:", average_vad[0])
    print("Arousal:", average_vad[1])
    print("Dominance:", average_vad[2])  

NameError: name 'csv' is not defined